# Projeto Final FTC - dez/2022 - Plataforma Fome Zero


### Contexto de negócio

A empresa Fome Zero é uma marketplace de restaurantes, que busca facilitar o encontro e negociações entre clientes e restaurantes. Os restaurantes fazem o cadastro dentro da plataforma da Fome Zero, que disponibilizainformações como endereço, tipo de culinária servida, se possui reservas, se faz
e ntregas e também uma nota de avaliação dos serviços e produtos do restaurante, dentre outras informações.

### O Desafio
O CEO precisa entender melhor o negócio para conseguir tomar as melhores decisões estratégicas e alavancar ainda mais a empresa Fome Zero, e para isso, ele precisa que seja feita uma análise nos dados da empresa e que sejam gerados dashboards, a partir dessas análises, para responder às perguntas.

### 0. Imports

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import folium
from haversine import haversine
import inflection

### 0.1 Loading Data

In [79]:
df = pd.read_csv('datasets/zomato.csv')
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,Botswana Pula(P),1,0,0,0,3,4.6,3F7E00,Excellent,619
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,"European, Asian",...,Botswana Pula(P),0,0,0,0,4,4.7,3F7E00,Excellent,469
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,"Filipino, American, Italian, Bakery",...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,867
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,Botswana Pula(P),0,0,0,0,3,4.4,5BA829,Very Good,858


In [80]:
df1 = df.copy()

### 0.2 Functions

In [4]:
df1.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [81]:
##Preenchimento do nome dos países
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
def country_name(country_id):
    return COUNTRIES[country_id]

##Criação do Tipo de Categoria de Comida
def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

# Criação do nome das Cores
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}

def color_name(color_code):
    return COLORS[color_code]

## Renomear as colunas do DataFrame
def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df

In [82]:
df1 = rename_columns(df1)

In [7]:
df1.columns


Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes'],
      dtype='object')

In [83]:
df1["cuisines"] = df1.loc[:, "cuisines"].astype(str).apply(lambda x: x.split(",")[0])
df1["color_level"] = df1.apply(lambda x: color_name(x['rating_color']), axis=1)
df1["tipo_comida"] = df1.apply(lambda x: create_price_tye(x['price_range']), axis=1)
df1["pais"] = df1.apply(lambda x: country_name(x['country_code']), axis =1)

In [84]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

In [85]:
df1

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,color_level,tipo_comida,pais
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,expensive,Philippines
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,expensive,Philippines
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,darkgreen,gourmet,Philippines
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,green,expensive,Philippines
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,green,expensive,Philippines
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7522,5912546,Eataly,208,İstanbul,"Zorlu Center AVM, Köprü Katı, Levazım Mahalles...","Zorlu Center AVM, Levazım, Beşiktaş","Zorlu Center AVM, Levazım, Beşiktaş, İstanbul",29.017326,41.065322,Italian,...,0,0,4,4.3,5BA829,Very Good,1367,green,gourmet,Turkey
7523,5913006,Tarihi Çınaraltı Aile Çay Bahçesi,208,İstanbul,"Çengelköy Mahallesi, Çınaraltı Camii Sokak, No...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052620,41.050280,Fast Food,...,0,0,2,4.5,3F7E00,Excellent,1172,darkgreen,normal,Turkey
7524,5923535,Boon Cafe & Restaurant,208,İstanbul,"Çengelköy Mahallesi, Çengelköy Caddesi, Kara S...",Çengelköy Merkez,"Çengelköy Merkez, İstanbul",29.052623,41.050717,Restaurant Cafe,...,0,0,4,4.2,5BA829,Very Good,1160,green,gourmet,Turkey
7525,5914190,Kanaat Lokantası,208,İstanbul,"Sultantepe Mahallesi, Selmani Pak Caddesi, No ...",Üsküdar Merkez,"Üsküdar Merkez, İstanbul",29.016590,41.025741,Home-made,...,0,0,3,4.0,5BA829,Very Good,770,green,expensive,Turkey


### 1. Descrição dos dados


### 1.1 Data Dimensions

In [11]:
print('Number of Rows {}'.format(df1.shape[0]))
print('Number of Rows {}'.format(df1.shape[1]))

Number of Rows 7527
Number of Rows 24


### 1.2 Data Types

In [12]:
df1.dtypes

restaurant_id             int64
restaurant_name          object
country_code              int64
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
color_level              object
tipo_comida              object
pais                     object
dtype: object

In [13]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

### 1.3 Check Na

In [14]:
df1.isna().sum()

restaurant_id           0
restaurant_name         0
country_code            0
city                    0
address                 0
locality                0
locality_verbose        0
longitude               0
latitude                0
cuisines                0
average_cost_for_two    0
currency                0
has_table_booking       0
has_online_delivery     0
is_delivering_now       0
switch_to_order_menu    0
price_range             0
aggregate_rating        0
rating_color            0
rating_text             0
votes                   0
color_level             0
tipo_comida             0
pais                    0
dtype: int64

### 1.4 Fillout NA

In [18]:
# Removendo os valores NA da coluna Cuisines
linhas_selecionadas = (df1['cuisines'] != 'NaN ') 
df1 = df1.loc[linhas_selecionadas, :].copy()

### 1.5 Change Types

(7527, 21)

### 1.6 Descriptive Statistical

#### 1.6.1 Numeric atributtes

In [15]:
num_attributes = df1.select_dtypes(include=['int64','float64'])
cat_attributes = df1.select_dtypes(exclude=['int64','float64','datetime64[ns]'])

In [16]:
# Central Tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# Dispersion (quão próximo ou longe da média): std, min, max, range, skew, kurtosis 
# skew (deformação em relação a curva normal)
# kusrtosis (concentração de dados)
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(min)).T
d3 = pd.DataFrame(num_attributes.apply(max )).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max() - x.min() )).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew() )).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis() )).T

# Concatenate
m = pd.concat([d2, d3, d4, ct1, ct2,d1, d5, d6]).T.reset_index()
m

# Rename columns
m.columns = ['atributtes', 'min', 'max', 'range','mean', 'median','std','skew','kurtosis']

In [17]:
m

,atributtes,min,max,range,mean,median,std,skew,kurtosis
0,restaurant_id,549.000000,1.904028e+07,1.903973e+07,1.055689e+07,7.701457e+06,7.074671e+06,-0.087788,-1.726571
1,country_code,1.000000,2.160000e+02,2.150000e+02,9.306536e+01,3.000000e+01,9.902465e+01,0.258687,-1.849127
2,longitude,-122.700455,1.753106e+02,2.980110e+02,3.376409e+01,7.378512e+01,7.739010e+01,-0.680775,-0.596714
3,latitude,-41.330428,5.597698e+01,9.730741e+01,1.933279e+01,2.524695e+01,2.325443e+01,-1.082988,0.617026
4,average_cost_for_two,0.000000,2.500002e+07,2.500002e+07,7.152113e+03,2.900000e+02,2.905875e+05,84.578959,7271.931662
5,has_table_booking,0.000000,1.000000e+00,1.000000e+00,6.217617e-02,0.000000e+00,2.414752e-01,3.626965,11.157839
6,has_online_delivery,0.000000,1.000000e+00,1.000000e+00,3.524645e-01,0.000000e+00,4.777377e-01,0.617766,-1.618796
7,is_delivering_now,0.000000,1.000000e+00,1.000000e+00,1.743058e-01,0.000000e+00,3.793723e-01,1.717359,0.949573
8,switch_to_order_menu,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000
9,price_range,1.000000,4.000000e+00,3.000000e+00,2.624552e+00,3.000000e+00,9.597301e-01,-0.040693,-0.983179


#### 1.6.2 Categorical atributtes

In [72]:
cat_attributes.apply(lambda x: x.unique().shape[0])

Restaurant Name     5914
City                 125
Address             6760
Locality            2272
Locality Verbose    2357
Cuisines             166
Currency              12
Rating color           7
Rating text           28
dtype: int64

### Geral

In [18]:
df1.columns


Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

In [19]:
#1. Quantos restaurantes únicos estão registrados? R: 6942
len(df1['restaurant_id'].unique())

6942

In [140]:
#2. Quantos países únicos estão registrados? R: 15
len(df1['country_code'].unique())

15

In [21]:
country_name(216)

'United States of America'

In [141]:
#3. Quantas cidades únicas estão registradas? R: 125
len(df1['city'].unique())

125

In [23]:
#4. Qual o total de avaliações feitas? R: 7527
df1['aggregate_rating'].count()

7527

In [24]:
#5. Qual o total de tipos de culinária registrados? R: 166
df1['cuisines'].nunique()

166

### País

In [396]:
#1. Qual o nome do país que possui mais cidades registradas? R: India: 49
df_unique = df1[['pais','city']].drop_duplicates()
df_unique[['pais','city']].groupby('pais').count().sort_values('city', ascending = False)

,city
pais,
India,49
United States of America,22
Philippines,12
South Africa,12
England,5
New Zeland,4
United Arab Emirates,4
Australia,3
Brazil,3


In [475]:
df1[['pais','city']].groupby('pais').nunique().reset_index().sort_values(by='city', ascending = False)

,pais,city
4,India,49
14,United States of America,22
7,Philippines,12
10,South Africa,12
3,England,5
6,New Zeland,4
13,United Arab Emirates,4
0,Australia,3
1,Brazil,3
2,Canada,3


In [398]:
# 2. Qual o nome do país que possui mais restaurantes registrados? R: India 3120
rest = df1[['pais','restaurant_id']].drop_duplicates()
rest[['pais','restaurant_id']].groupby('pais').count().sort_values('restaurant_id', ascending = False)

,restaurant_id
pais,
India,3120
United States of America,1378
England,400
South Africa,346
United Arab Emirates,300
Brazil,240
New Zeland,239
Australia,180
Canada,180


In [158]:
df_aux = df1.loc[:, ['restaurant_id', 'pais']].groupby(['pais']).max()
df_aux = df_aux.sort_values('restaurant_id', ascending = False).reset_index()
df_aux.loc[0, 'pais']

'India'

In [191]:
# 3. Qual o nome do país que possui mais restaurantes com o nível de preço igual a 4 registrados? R: United States of America = 420
cols = ['pais', 'restaurant_id','price_range']
df1.loc[:,cols].groupby([ 'pais','price_range']).count().sort_values(['restaurant_id'],ascending=False)

restaurant_id
pais                     price_range               
India                    2                     1613
                         3                     1022
                         1                      662
United States of America 2                      479
                         4                      420
                         3                      329
South Africa             4                      237
India                    4                      210
England                  3                      181
United States of America 1                      167
United Arab Emirates     3                      164
Brazil                   4                      159
England                  2                      140
United Arab Emirates     4                      122
South Africa             3                      108
New Zeland               3                      106
England                  4                       83
New Zeland               4                       81
Australia                3                       72
Turkey                   3                       71
Australia                2                       70
Qatar                    4                       63
Turkey                   4                       62
Philippines              3                       61
Singapure                4                       59
Canada                   2                       58
Indonesia                3                       57
Brazil                   3                       55
Canada                   4                       54
New Zeland               2                       50
Canada                   3                       45
Sri Lanka                3                       43
United Arab Emirates     2                       39
Australia                4                       33
England                  1                       33
South Africa             2                       32
Turkey                   2                       31
Brazil                   2                       29
Qatar                    3                       27
Canada                   1                       23
Philippines              4                       23
Sri Lanka                2                       23
Indonesia                4                       18
Brazil                   1                       18
Sri Lanka                4                       18
Singapure                3                       18
New Zeland               1                       16
United Arab Emirates     1                        9
Indonesia                2                        7
Australia                1                        5
South Africa             1                        5
Singapure                2                        5
Qatar                    2                        4
Philippines              2                        4
Sri Lanka                1                        3
Turkey                   1                        1

In [305]:
df1.loc[df1['price_range'] == 4, ['pais','restaurant_id']].groupby(['pais']).count().reset_index().sort_values('restaurant_id', ascending=False)

,pais,restaurant_id
14,United States of America,420
10,South Africa,237
4,India,210
1,Brazil,159
13,United Arab Emirates,122
3,England,83
6,New Zeland,81
8,Qatar,63
12,Turkey,62
9,Singapure,59


In [406]:
#4. Qual o nome do país que possui a maior quantidade de tipos de culinária distintos? R: India
#cols = ['Cuisines', 'pais']
df_cuisines_unique = df1[['pais','cuisines']].drop_duplicates()
df_cuisines_unique[['pais','cuisines']].groupby(['pais']).count().sort_values(['cuisines'],ascending=False)


,cuisines
pais,
India,78
United States of America,75
England,52
United Arab Emirates,46
Australia,43
Brazil,43
Canada,42
South Africa,42
New Zeland,37


In [407]:
# 5. Qual o nome do país que possui a maior quantidade de avaliações feitas? R: India: 3507
df1[['votes', 'pais']].groupby(['pais']).count().sort_values(['votes'],ascending=False)

,votes
pais,
India,3507
United States of America,1395
England,437
South Africa,382
United Arab Emirates,334
Brazil,261
New Zeland,253
Australia,180
Canada,180


In [92]:
# 6. Qual o nome do país que possui a maior quantidade de restaurantes que fazem entrega? R: India, id=2365

df_aux = df1.loc[:, ['has_online_delivery', 'pais']].groupby(['pais']).max()
df_aux = df_aux.sort_values('has_online_delivery', ascending = False).reset_index()
df_aux.loc[0, 'pais']

'India'

In [319]:
df1.loc[df1['has_online_delivery'] == 1, ['pais','restaurant_id']].groupby(['pais']).count().reset_index().sort_values('restaurant_id', ascending=False)

,pais,restaurant_id
0,India,2365
3,United Arab Emirates,236
2,Qatar,43
1,Philippines,9


In [95]:
# 7. Qual o nome do país que possui a maior quantidade de restaurantes que aceitam reservas? R: INdia (291)
df_aux = df1.loc[:, ['has_table_booking', 'pais', 'restaurant_id']].groupby(['pais']).max()
df_aux = df_aux.sort_values('restaurant_id', ascending = False).reset_index()
df_aux.loc[0, 'pais']

'India'

In [412]:
df1.loc[df1['has_table_booking'] == 1,['pais','restaurant_id']].groupby(['pais']).count().reset_index().sort_values('restaurant_id', ascending=False)

,pais,restaurant_id
2,India,291
1,England,58
0,Australia,29
5,Philippines,28
4,New Zeland,19
3,Indonesia,14
10,United Arab Emirates,12
6,Qatar,5
9,Turkey,5
7,South Africa,3


In [89]:
# 8. Qual o nome do país que possui, na média, a maior quantidade de avaliações registrada? R: Indonesia (1115.317073)

df_aux = df1.loc[:, ['votes', 'pais']].groupby(['pais']).mean()
df_aux = df_aux.sort_values('votes', ascending = False).reset_index()
df_aux.loc[0, 'pais']

'Indonesia'

In [320]:
df1[['votes', 'pais']].groupby(['pais']).mean().reset_index().sort_values('votes', ascending=False)

,pais,votes
5,Indonesia,1115.317073
4,India,905.361848
0,Australia,724.205556
12,Turkey,643.103030
13,United Arab Emirates,614.296407
7,Philippines,600.284091
2,Canada,583.433333
14,United States of America,381.463082
8,Qatar,374.904255
10,South Africa,236.023560


In [96]:
# 9. Qual o nome do país que possui, na média, a maior nota média registrada? R: Indonesia (4.600000)
df_aux = df1.loc[:, ['aggregate_rating', 'pais']].groupby(['pais']).mean()
df_aux = df_aux.sort_values('aggregate_rating', ascending = False).reset_index()
df_aux.loc[0, 'pais']

'Indonesia'

In [294]:
df1[['aggregate_rating', 'pais']].groupby(['pais']).mean().reset_index().sort_values('aggregate_rating', ascending=False)

,pais,aggregate_rating
5,Indonesia,4.600000
7,Philippines,4.462500
9,Singapure,4.429268
14,United States of America,4.404659
0,Australia,4.372778
2,Canada,4.321667
12,Turkey,4.313939
8,Qatar,4.267021
6,New Zeland,4.166798
11,Sri Lanka,4.070115


In [98]:
#10. Qual o nome do país que possui, na média, a menor nota média registrada? R: Brazil
df_aux = df1.loc[:, ['aggregate_rating', 'pais']].groupby(['pais']).mean()
df_aux = df_aux.sort_values('aggregate_rating', ascending = True).reset_index()
df_aux.loc[0, 'pais']

'Brazil'

In [293]:
df1[['aggregate_rating', 'pais']].groupby(['pais']).mean().reset_index().sort_values('aggregate_rating', ascending=True)

,pais,aggregate_rating
1,Brazil,3.349425
4,India,4.017479
3,England,4.041190
10,South Africa,4.054188
13,United Arab Emirates,4.055689
11,Sri Lanka,4.070115
6,New Zeland,4.166798
8,Qatar,4.267021
12,Turkey,4.313939
2,Canada,4.321667


In [291]:
# 11. Qual a média de preço de um prato para dois por país?
df1[['average_cost_for_two', 'pais']].groupby(['pais']).mean().reset_index().sort_values('average_cost_for_two', ascending=False)


,pais,average_cost_for_two
5,Indonesia,312682.926829
0,Australia,138959.783333
11,Sri Lanka,2607.471264
7,Philippines,1211.090909
4,India,702.303964
10,South Africa,335.479058
8,Qatar,176.648936
13,United Arab Emirates,158.218563
9,Singapure,141.036585
1,Brazil,137.375479


### Cidade

In [324]:
#1. Qual o nome da cidade que possui mais restaurantes registrados? R: Istanbul
df_aux = df1.loc[:, ['restaurant_id', 'city']].groupby(['city']).count()
df_aux = df_aux.sort_values('restaurant_id', ascending = False).reset_index()
df_aux.loc[0, 'city']

'Nagpur'

In [322]:
df1[['restaurant_id', 'city']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False)

,city,restaurant_id
76,Nagpur,99
111,Surat,95
14,Birmingham,95
0,Abu Dhabi,95
32,Doha,94
...,...,...
40,Ghaziabad,1
74,Muntinlupa City,1
68,Marikina City,1
123,Zirakpur,1


In [416]:
#2. Qual o nome da cidade que possui mais restaurantes com nota média acima de 4? Bangalore - 86
df1.loc[df1['aggregate_rating'] > 4, ['restaurant_id','city']].groupby(['city']).count().reset_index().sort_values('restaurant_id', ascending=False)

,city,restaurant_id
11,Bangalore,86
43,Houston,82
23,Chennai,82
56,London,81
53,Kolkata,80
...,...,...
96,Roodepoort,1
37,Ghaziabad,1
75,Nasik,1
119,Zirakpur,1


In [417]:
#3. Qual o nome da cidade que possui mais restaurantes com nota média abaixo de 2.5? Gangtok - 47
df1.loc[df1['aggregate_rating'] < 2.5, ['restaurant_id','city']].groupby(['city']).count().reset_index().sort_values('restaurant_id', ascending=False)

,city,restaurant_id
6,Gangtok,47
10,Ooty,22
16,São Paulo,17
1,Brasília,15
14,Rio de Janeiro,13
8,Manchester,7
2,Clarens,4
15,Shimla,4
13,Puducherry,3
5,Edinburgh,2


In [205]:
#4. Qual o nome da cidade que possui o maior valor médio de um prato para dois? R: Adelaide
df_aux = df1.loc[:, ['average_cost_for_two', 'city', 'price_range']].groupby(['city']).mean()
df_aux = df_aux.sort_values('average_cost_for_two', ascending = False).reset_index()
df_aux.loc[0, 'city']

'Adelaide'

In [422]:
df1[['average_cost_for_two', 'city']].sort_values('average_cost_for_two', ascending=False)

,average_cost_for_two,city
385,25000017,Adelaide
6050,1200000,Jakarta
6049,1200000,Jakarta
6086,700000,Jakarta
6044,600000,Jakarta
...,...,...
2372,0,Fujairah
1897,0,San Diego
1517,0,Ottawa
166,0,Brasília


In [428]:
df1.where(df1['average_cost_for_two'] == 25000017).dropna()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,color_level,tipo_comida,pais
385,16608070.0,d'Arry's Verandah Restaurant,14.0,Adelaide,"Osborn Rd, McLaren Vale",McLaren Vale,McLaren Vale,138.545242,-35.198372,Modern Australian,...,0.0,0.0,1.0,4.7,3F7E00,Excellent,203.0,darkgreen,cheap,Australia


In [333]:
#5. Qual o nome da cidade que possui a maior quantidade de tipos de culinária distintas? R: Nagpur: 99
df_aux = df1.loc[:, ['cuisines', 'city']].groupby(['city']).count()
df_aux = df_aux.sort_values('cuisines', ascending = False).reset_index()
df_aux.loc[0, 'city']

'Nagpur'

In [335]:
df1[['cuisines', 'city']].groupby('city').count().reset_index().sort_values('cuisines', ascending=False)

,city,cuisines
76,Nagpur,99
111,Surat,95
14,Birmingham,95
0,Abu Dhabi,95
32,Doha,94
...,...,...
40,Ghaziabad,1
74,Muntinlupa City,1
68,Marikina City,1
123,Zirakpur,1


In [208]:
# 6. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem reservas? Bangalore: 47

'Nagpur'

In [434]:
df1.loc[df1['has_table_booking'] == 1, ['city','restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False)

,city,restaurant_id
8,Bangalore,47
16,Chennai,35
55,Pune,31
44,Mumbai,23
48,New Delhi,17
...,...,...
11,Bogor,1
12,Boston,1
33,Johannesburg,1
47,Nashik,1


In [211]:
# 7. Qual o nome da cidade que possui a maior quantidade de restaurantes que fazem entregas?
df_aux = df1.loc[:, ['restaurant_id', 'city','is_delivering_now']].groupby(['city']).count()
df_aux = df_aux.sort_values('is_delivering_now', ascending = False).reset_index()
df_aux.loc[0, 'city']


'Nagpur'

In [435]:

df1.loc[df1['is_delivering_now'] == 1, ['city','restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False)

,city,restaurant_id
5,Aurangabad,53
48,Vadodara,52
26,Ludhiana,50
7,Bhopal,50
21,Jaipur,49
12,Dehradun,48
4,Amritsar,48
2,Ahmedabad,47
49,Varanasi,43
42,Ranchi,43


In [ ]:
# 8. Qual o nome da cidade que possui a maior quantidade de restaurantes que aceitam pedidos online?
# Abu Dhabi 86

In [436]:
df1.loc[df1['has_online_delivery'] == 1, ['city','restaurant_id']].groupby('city').count().reset_index().sort_values('restaurant_id', ascending=False)

,city,restaurant_id
0,Abu Dhabi,86
7,Bhopal,81
45,Sharjah,80
33,Nagpur,80
49,Vadodara,79
5,Aurangabad,76
40,Patna,74
3,Allahabad,74
46,Surat,73
25,Kolkata,73


### Restaurantes

In [192]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

In [438]:
df1.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,color_level,tipo_comida,pais
0,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,expensive,Philippines
1,6310675,Mama Lou's Italian Kitchen,162,Las Piñas City,"Block 1, Lot 36, Tropical Avenue Corner Tropic...",BF International,"BF International, Las Piñas City",121.009787,14.447615,Italian,...,0,0,3,4.6,3F7E00,Excellent,619,darkgreen,expensive,Philippines
2,6314542,Blackbird,162,Makati City,"Nielson Tower, Ayala Triangle Gardens, Salcedo...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.024562,14.556042,European,...,0,0,4,4.7,3F7E00,Excellent,469,darkgreen,gourmet,Philippines
3,6301293,Banapple,162,Makati City,"Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...","Ayala Triangle Gardens, Salcedo Village, Makat...",121.023171,14.556196,Filipino,...,0,0,3,4.4,5BA829,Very Good,867,green,expensive,Philippines
4,6315689,Bad Bird,162,Makati City,"Hole In The Wall, Floor 4, Century City Mall, ...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027708,14.565899,American,...,0,0,3,4.4,5BA829,Very Good,858,green,expensive,Philippines


In [481]:
# 1. Qual o nome do restaurante que possui a maior quantidade de avaliações? R: 3928    Bawarchi
df1[['votes', 'restaurant_name']].sort_values(by='votes', ascending=False)

,votes,restaurant_name
3928,41333,Bawarchi
3038,17394,Byg Brewski Brewing Company
2986,15270,Toit
3002,14984,Truffles
5076,13627,Hauz Khas Social
...,...,...
219,0,Le Bistrot du Cuisinier
3629,0,Blue Restaurant
285,0,Cantinho da Gula
6752,0,The Pasta Factory


In [477]:
df1.loc[df1['votes']== df1['votes'].max(), 'restaurant_name']

3928    Bawarchi
Name: restaurant_name, dtype: object

In [283]:
#R: Indian Grill Room (id = 7528.0)
# 2. Qual o nome do restaurante com a maior nota média?

df1[['restaurant_id', 'restaurant_name','aggregate_rating']].groupby('restaurant_name').mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_name,restaurant_id,aggregate_rating
2436,Indian Grill Room,7528.0,4.9
4156,Restaurant Mosaic @ The Orient,75989.0,4.9
4848,Tapri Central,101212.0,4.9
3982,Pousada By The Beach,130664.0,4.9
5165,The Huddle Sports Bar and Grill - Citymax Hote...,203518.0,4.9
...,...,...,...
4665,Sree Sagar Restaurantat,18989652.0,0.0
5588,V. Krishnan Bakery,18994548.0,0.0
4146,Republic of Chicken,19034826.0,0.0
1249,City Point,19035941.0,0.0


In [234]:
# 3. Qual o nome do restaurante que possui o maior valor de um prato para duas pessoas? R: d'Arry's Verandah Restaurant
df1[['average_cost_for_two', 'restaurant_name']].groupby('restaurant_name').max().reset_index().sort_values('average_cost_for_two', ascending=False)



,restaurant_name,average_cost_for_two
5896,d'Arry's Verandah Restaurant,25000017
5011,The Café - Hotel Mulia,1200000
1915,GIA Restaurant & Bar,700000
4283,SKYE,600000
4440,Shabu-Shabu Shaburi,600000
...,...,...
3401,Mr Go's,0
5720,Whalesbone Oyster House,0
5421,Tiny Boxwood's,0
4188,Ristorantino,0


In [ ]:
# 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?
#R: Loca Como tu Madre (id = 6600100)

In [280]:
df1.loc[df1['cuisines'] == 'Brazilian', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name','restaurant_id']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_name,restaurant_id,aggregate_rating
48,Loca Como tu Madre,6600100,0.0
18,Café Savana,6600119,0.0
45,Inácia Poulet Rôti,6600124,0.0
39,Galeteria Beira Lago,6600456,0.0
63,Severina,6601251,0.0
...,...,...,...
34,Fogo de Chao Brazilian Steakhouse,16881352,4.8
17,Braseiro da Gávea,7300955,4.9
2,Aprazível,7302898,4.9
69,Texas de Brazil,18403563,4.9


In [368]:
# Brazilian
df1['cuisines'].unique()

array(['Italian', 'European', 'Filipino', 'American', 'Korean', 'Pizza',
       'Taiwanese', 'Japanese', 'Coffee', 'Chinese', 'Seafood',
       'Singaporean', 'Vietnamese', 'Latin American', 'Healthy Food',
       'Cafe', 'Fast Food', 'Brazilian', 'Argentine', 'Arabian', 'Bakery',
       'Tex-Mex', 'Bar Food', 'International', 'French', 'Steak',
       'German', 'Sushi', 'Grill', 'Peruvian', 'North Eastern',
       'Ice Cream', 'Burger', 'Mexican', 'Vegetarian', 'Contemporary',
       'Desserts', 'Juices', 'Beverages', 'Spanish', 'Thai', 'Indian',
       'Mineira', 'BBQ', 'Mongolian', 'Portuguese', 'Greek', 'Asian',
       'Author', 'Gourmet Fast Food', 'Lebanese', 'Modern Australian',
       'African', 'Coffee and Tea', 'Australian', 'Middle Eastern',
       'Malaysian', 'Tapas', 'New American', 'Pub Food', 'Southern',
       'Diner', 'Donuts', 'Southwestern', 'Sandwich', 'Irish',
       'Mediterranean', 'Cafe Food', 'Korean BBQ', 'Fusion', 'Canadian',
       'Breakfast', 'Cajun', 'Ne

In [484]:
mean = df1.loc[df1['cuisines']=='Brazilian', 'aggregate_rating']
mean.mean()

3.47

In [453]:
#5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação? R: Brazeiro da Gávea (id=7300955.0) nota 4.9  sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])
df1_cuisines = df1.where(df1['cuisines'] == 'Brazilian')
df1_cuisines = df1_cuisines.where(df1['pais'] == 'Brazil')
df1_cuisines = df1_cuisines.dropna()
df1_cuisines[['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True])

,restaurant_name,restaurant_id,aggregate_rating
12,Braseiro da Gávea,7300955.0,4.90
2,Aprazível,7302898.0,4.90
39,Mocotó,6709740.0,4.80
26,Fogo de Chão,7301525.0,4.80
52,Zazá Bistrô Tropical,7300483.0,4.70
1,Aconchego Carioca,7302859.0,4.70
51,Veloso,6703176.0,4.60
29,Garota de Ipanema,7301107.5,4.55
22,Esquina Mocotó,6709580.0,4.50
30,Giuseppe Grill Mar,7301112.0,4.50


In [252]:
df1.columns


Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

In [339]:
#6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas? R: não
df1[['has_online_delivery', 'restaurant_name', 'votes']].groupby('restaurant_name').mean().reset_index().sort_values('votes', ascending=False)

,restaurant_name,has_online_delivery,votes
5437,Toit,0.0,15270.000000
5500,Truffles,0.0,14984.000000
497,Bawarchi,1.0,13903.666667
2185,Hauz Khas Social,1.0,13627.000000
4443,Shah Ghouse Hotel & Restaurant,1.0,11836.000000
...,...,...,...
5659,Vinheria Santa Clara,0.0,0.000000
4899,Tempero das Gerais,0.0,0.000000
1031,Cantinho da Gula,0.0,0.000000
191,Alston Bar & Beef,0.0,0.000000


In [456]:
# 7. Os restaurantes que fazem reservas são também, na média, os restaurantes que possuem o maior valor médio de um prato para duas pessoas? Sim
df1[['has_table_booking','average_cost_for_two']].groupby(['has_table_booking']).mean().sort_values('average_cost_for_two', ascending=False)

,average_cost_for_two
has_table_booking,
1,62931.512821
0,3454.031591


In [338]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

In [463]:
# Sim
#8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)?
# making boolean series for a team name
filter1 = df1["pais"]=="United States of America"
  
# making boolean series for age
filter2 = df1["cuisines"] == "Japanese"
filter3 = df1["cuisines"] == "BBQ"
  
# filtering data on basis of both filters
df_eua = df1.where(filter1 & (filter2 | filter3)).dropna()
df_eua[['cuisines','average_cost_for_two']].groupby(['cuisines']).mean().sort_values('average_cost_for_two', ascending=False)

,average_cost_for_two
cuisines,
Japanese,56.406250
BBQ,39.642857


In [ ]:
eua =df1['pais']== 'United States America']
japonesa =df1['cuisines']== 'japanese']

aux1 = df1.loc[eua & japonesa,]
aux1['average_cost_for_two'].mean()  ## filtragem

bbq = df1['cuisines'] =='BBQ'
df1.loc[bbq,'average_cost_for_two'].mean()

### Tipos de Culinária

In [346]:
# R Cafe Del Sol Classico - 6501298    .sort_values(['aggregate_rating','restaurant_id'], ascending=[False,True])
#1. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a maior média de avaliação?


In [347]:
### Importante: como colocar dois tipos de ordenação
df1.loc[df1['cuisines'] == 'Italian', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_name,restaurant_id,aggregate_rating
44,Cafe Del Sol Classico,6501298.0,4.9
188,Ombra,7100171.0,4.9
61,Celino's,7700796.0,4.9
12,Andre's Cucina & Polenta Bar,16587684.0,4.9
85,Di Rienzo Grocery & Deli,16663419.0,4.9
...,...,...,...
154,Le Delicatezze Di Bruno,6800666.0,0.0
30,Bene - Sheraton Rio Hotel,7304910.0,0.0
141,La Bocca Bar e Trattoria,18334183.0,0.0
298,The Pasta Factory,18756343.0,0.0


In [348]:
# 2. Dos restaurantes que possuem o tipo de culinária italiana, qual o nome do restaurante com a menor média de avaliação?
#R: Avenida Paulista, ID: 6600542.0
df1.loc[df1['cuisines'] == 'Italian', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_name,restaurant_id,aggregate_rating
17,Avenida Paulista,6600542.0,0.0
250,Ristorantino,6714499.0,0.0
227,Più,6715707.0,0.0
154,Le Delicatezze Di Bruno,6800666.0,0.0
30,Bene - Sheraton Rio Hotel,7304910.0,0.0
...,...,...,...
63,Cerroni's Purple Garlic,17156258.0,4.9
121,Guillermo's,17156955.0,4.9
62,Central Grocery,17242351.0,4.9
337,Zolocrust - Hotel Clarks Amer,18209498.0,4.9


In [349]:
# 3. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a maior média de avaliação?
# OEB Breakfast Co. (id: 16631515.0)
df1.loc[df1['cuisines'] == 'American', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_name,restaurant_id,aggregate_rating
270,OEB Breakfast Co.,16631515.0,4.9
290,Portillo's Hot Dogs,16737455.0,4.9
321,S'MAC,16776778.0,4.9
4,5 Napkin Burger,16782050.0,4.9
141,Fat Cat,16799018.0,4.9
...,...,...,...
227,Lava Lounge,18692865.0,3.3
421,Thunderbird Cafe,7100971.0,3.2
53,Bob's,7303748.0,2.6
178,Guy Fieri's Kitchen & Bar,18445965.0,2.2


In [350]:
#4. Dos restaurantes que possuem o tipo de culinária americana, qual o nome do restaurante com a menor média de avaliação?
#R: Alston Bar & Beef    id: 18756337.0
df1.loc[df1['cuisines'] == 'American', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_name,restaurant_id,aggregate_rating
13,Alston Bar & Beef,18756337.0,0.0
178,Guy Fieri's Kitchen & Bar,18445965.0,2.2
53,Bob's,7303748.0,2.6
421,Thunderbird Cafe,7100971.0,3.2
269,O'Learys,18245065.0,3.3
...,...,...,...
402,The Observatory,17048908.0,4.9
190,Hodad's,17069832.0,4.9
218,Kono's Surf Club Cafe,17070293.0,4.9
397,The Magnolia Pancake Haus,17153940.0,4.9


In [351]:
# 5. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a maior média de avaliação?
#R: Mandi@36 id: 18535007.0
df1.loc[df1['cuisines'] == 'Arabian', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_name,restaurant_id,aggregate_rating
23,Mandi@36,18535007.0,4.70
22,Madfoon Al Khaimah,5600517.0,4.50
37,Wok of Fame,6200166.0,4.50
34,Three Kings,18770281.0,4.50
8,Aroos Damascus,2901582.0,4.40
10,Azkadenya,9318540.5,4.30
9,Awani,207265.0,4.20
32,Sheba Kuzhimandhi,902109.0,4.20
28,Rotisseria Sírio Libaneza,7304636.0,4.20
24,Marsala Food Company,18683886.0,4.20


In [352]:
# 6. Dos restaurantes que possuem o tipo de culinária árabe, qual o nome do restaurante com a menor média de avaliação?
# Raful id: 6706513.0
df1.loc[df1['cuisines'] == 'Arabian', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_name,restaurant_id,aggregate_rating
26,Raful,6706513.0,0.00
18,Empório Árabe,6601535.0,2.70
29,Salim Sou Eu,6600547.0,3.10
35,V Empire,18698592.0,3.20
1,Abad Hot Chicken,18698769.0,3.20
25,Murjan - Nour Arjaan by Rotana,16540460.0,3.30
15,Bon Appetit,16541428.0,3.30
21,Libanus,6600939.0,3.40
16,Breeze - Radisson Blu Resort,16540483.0,3.40
5,Al Nokhada - Radisson Blu Resort,16540485.0,3.40


In [353]:
#7. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome dorestaurante com a maior média de avaliação?
#R: Sushi Samba id:6107336.0
df1.loc[df1['cuisines'] == 'Japanese', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_name,restaurant_id,aggregate_rating
130,Sushi Samba,6107336.0,4.9
9,Chotto Matte,6116563.0,4.9
91,Ramen Yushoken,6309831.0,4.9
127,Sushi Leblon,7300004.0,4.9
58,Mendokoro Ramenba,12505776.0,4.9
...,...,...,...
128,Sushi Loko,6600379.0,2.9
25,Haná,6600314.0,2.7
23,Hachiko,7304077.0,2.5
84,Orienthai,7300930.0,2.4


In [354]:
#8. Dos restaurantes que possuem o tipo de culinária japonesa, qual o nome dorestaurante com a menor média de avaliação?
#R: Banzai Sushi id: 6600203.0
df1.loc[df1['cuisines'] == 'Japanese', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_name,restaurant_id,aggregate_rating
5,Banzai Sushi,6600203.0,0.0
84,Orienthai,7300930.0,2.4
23,Hachiko,7304077.0,2.5
25,Haná,6600314.0,2.7
128,Sushi Loko,6600379.0,2.9
...,...,...,...
58,Mendokoro Ramenba,12505776.0,4.9
76,Nobu Perth,16598168.0,4.9
102,Samurai,16924138.0,4.9
73,Noble Fish,16989399.0,4.9


In [355]:
# 9. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a maior média de avaliação?
#R: Kanaat Lokantası   id: 5914190.0
df1.loc[df1['cuisines'] == 'Home-made', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,restaurant_name,restaurant_id,aggregate_rating
1,Kanaat Lokantası,5914190.0,4.0
0,GurMekan Restaurant,6007184.0,3.7


In [357]:
# 10. Dos restaurantes que possuem o tipo de culinária caseira, qual o nome do restaurante com a menor média de avaliação?
#R: GurMekan Restaurant  id:6007184.0
df1.loc[df1['cuisines'] == 'Home-made', ['restaurant_id','restaurant_name', 'aggregate_rating']].groupby(['restaurant_name']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[True, True])

,restaurant_name,restaurant_id,aggregate_rating
0,GurMekan Restaurant,6007184.0,3.7
1,Kanaat Lokantası,5914190.0,4.0


In [466]:
# 11. Qual o tipo de culinária que possui o maior valor médio de um prato para duas pessoas?
#R: Modern Australian, valor: 25000017
df_cuisines = df1[['cuisines', 'average_cost_for_two']].groupby(['cuisines']).mean().reset_index().sort_values('average_cost_for_two', ascending=False)
df_cuisines

,cuisines,average_cost_for_two
104,Modern Australian,1.470693e+06
162,Western,3.333333e+05
75,Indonesian,2.000000e+05
148,Sunda,2.000000e+05
46,Dimsum,1.500000e+05
...,...,...
108,Moroccan,1.633333e+01
134,Russian,1.500000e+01
117,Others,1.000000e+01
126,Polish,1.000000e+01


In [363]:
#12. Qual o tipo de culinária que possui a maior nota média? R: Others
df1[['cuisines', 'aggregate_rating', 'restaurant_id']].groupby(['cuisines']).mean().reset_index().sort_values(['aggregate_rating', 'restaurant_id'], ascending=[False, True])

,cuisines,aggregate_rating,restaurant_id
117,Others,4.900000,1.663458e+07
130,Ramen,4.800000,1.144792e+07
118,Ottoman,4.800000,6.003596e+06
53,Egyptian,4.800000,1.874113e+07
148,Sunda,4.750000,7.417452e+06
...,...,...,...
18,Bengali,2.133333,1.829160e+07
0,Afghan,2.050000,1.066796e+07
165,nan,1.780000,1.737961e+07
49,Drinks Only,0.000000,1.654752e+07


In [469]:
# 13. Qual o tipo de culinária que possui mais restaurantes que aceitam pedidos online e fazem entregas?
# North Indian
filter1 = df1["is_delivering_now"]== 1
filter2 = df1["has_online_delivery"] == 1
  
# filtering data on basis of both filters
df_culinaria = df1.where(filter1 & filter2).dropna()

df_culinaria[['cuisines','restaurant_id']].groupby(['cuisines']).count().sort_values('restaurant_id', ascending=False)

,restaurant_id
cuisines,
North Indian,343
Cafe,142
South Indian,84
Burger,83
Fast Food,76
Pizza,64
Chinese,52
Continental,40
Italian,40


In [358]:
df1.columns

Index(['restaurant_id', 'restaurant_name', 'country_code', 'city', 'address',
       'locality', 'locality_verbose', 'longitude', 'latitude', 'cuisines',
       'average_cost_for_two', 'currency', 'has_table_booking',
       'has_online_delivery', 'is_delivering_now', 'switch_to_order_menu',
       'price_range', 'aggregate_rating', 'rating_color', 'rating_text',
       'votes', 'color_level', 'tipo_comida', 'pais'],
      dtype='object')

In [471]:
# Variavel local
def conta(b,a):
    c=b*a
    print (c)
    return (a-b)
a=4
b=5
c=6
conta(a,b)

20


1

In [ ]:
'Restaurant ID'.lower().replace(' ', '_')

In [ ]:
def rename columns(data):
    df = data.copy()
    old_columns = df.columns ##(salvando a lista com o nome dasd colunas)
    new_cols=[]
    
    for col in old_cols:
        new = col.lower().replace(' ', '_')
        new_cols.append(new)
        
    df.columns=new_cols
    return df

In [ ]:
rename_columns(data)

In [ ]:
data=rename_columns(data)
data.columns

In [ ]:
# Revisando se tem na e duplicados
data.isna().sum()

data.dropna(inplace=True)  ## sobrescreve a variavel original

data.duplicated() ## Para saber se tem duplicados
data.drop_duplicates(inplace=True)

In [ ]:
## Função country name
def country_name(x):
    country_name = {
        1: 'India',
        14: 'australia',
        30: 'Brasil'
    }
    return country_name[x]

In [ ]:
country_name[1]

In [ ]:
## Esse x é variável global)... apply serve para aplicar qquer função, 
#o lambda percorre todo dataset e entrega a informação que quero
#apply lambda de x, aplica linha a linha
data['country_name']=data['country_code'].apply(lambda x: country_name(x))

In [ ]:
## Esse x é local
data['cuisines'] = data['cuisines'].apply(lambda x: x.split(',')[0]) ## separa e cria uma lista, 
#retorna o primeiro indice (é isso que faz o zeero)

In [ ]:
# apply com duas colunas - função de conversão de moeda
#verifica e salva a condição dentro da coluna nova que criou

data['currency'].unique()
data['price_dolar']=data[['currency', average_cost_two']].apply(lambda x: (x['average_cost_for_two']/12,82) if x['currency' == 'Botswana Pula (P)' else
                          (x['average_cost_for_two']/5,31) if x['currency' == ''Brazilian (R$)' else
                                                                                            else 0, axis=1)
                          

In [473]:
x=11
if x <12:
    print('menor')
elif x ==12:
    print('igual')
else:
    prit('maior')

menor


In [ ]:
np.round(data.describe().T,2)

In [ ]:
dta.hist(bins=25);

In [ ]:
data = data.loc[(data['price_dolar']< 25000000) & (data['votes'] <430000)]

In [ ]:
data['price_dolar'].max()

In [485]:
comidas = ['feijoada', 'feijao']

In [486]:
len(comidas)

2

In [487]:
def a(b):
    c=b*b
    return c

b=5
c=7

print(a(c))

49


In [490]:
nome=['Meigaron']
idade=30
altura=1.85
comida_favorita='bis'

print(type(nome))
print(type(idade))
print(type(altura))
print(type(comida_favorita))

<class 'list'>
<class 'int'>
<class 'float'>
<class 'str'>


In [492]:
usuarios = {'nome': 'fernanda',
            'idade':25,
            'estudante':True}

In [495]:
usuarios.values()

dict_values(['fernanda', 25, True])

In [496]:
def equacao1(a):
    x=10
    print(a+x)
    return(a)

def equacao2(a):
    c=13
    print(a+c)
    return (c*a)

b=3
c=equacao1(b)
d=equacao2(c)
print(d)

13
16
39


In [500]:
usuarios={'pedro':75, 'joao': 23, 'marcos':50, 'ana':48, 'pamela':85, 'camile':40}

aprovados=[]
reprovados=[]

for nome,nota in usuarios.items():
    if nota >= 50:
        aprovados.append(nome)
    else:
        reprovados.append(nome)
print('aprovados: ')

for aluno in aprovados:
    print(aluno)
        

aprovados: 
pedro
marcos
pamela


In [501]:
df1.shape

(7527, 24)

In [504]:
df1['pais'].unique()

array(['Philippines', 'Brazil', 'Australia', 'United States of America',
       'Canada', 'Singapure', 'United Arab Emirates', 'India',
       'Indonesia', 'New Zeland', 'England', 'Qatar', 'South Africa',
       'Sri Lanka', 'Turkey'], dtype=object)